# Testing notebooks

> _The f🥇rst test is the hardest to write._

Tests are investments and testing over time measures the return on investment.  Testing promotes

* Longevity
* Protection from upstream changes.  
* Value to you and consumers of your software.

Often authors defer to notebooks to experiment and test computational ideas.  There are testing tools for complete notebooks like _informal_ tests with nbconvert or _formal_ tests with nbval.  These tests approaches apply tests to static documents outside of the interactive computing context.

> **_Remember_ 💭 [Restart and run all or it didn't happen](2018-07-16-Testing-restart-run-all.ipynb).**

This essay discusses using formal testing tools during interactive computing contexts in modern computational notebooks.  These tools include: [__doctest__](https://docs.python.org/3/library/doctest.html), [__unittest__](https://docs.python.org/3/library/unittest.html), and [__pytest__](https://doc.pytest.org/).

---

This topic has been discussed in some of our past presentations.

* https://github.com/deathbeds/LitAF
* https://github.com/deathbeds/nostalgiaforever
* https://github.com/deathbeds/wtf

## Test Assertions

This document commonly uses `assert` statements to indicate a test is happening.  Assertions are useful in testing, but not in production.  [`assert` statements can introduce potential security vulnerabilites](https://hackernoon.com/10-common-security-gotchas-in-python-and-how-to-avoid-them-e19fbe265e03#91f9) for optimized python code.

> [Only use assert statements to communicate with other developers, such as in unit tests or in to guard against incorrect API usage.](https://hackernoon.com/10-common-security-gotchas-in-python-and-how-to-avoid-them-e19fbe265e03#91f9)

## Doctest

[`doctest`](https://docs.python.org/3/library/doctest.html) is the simplest Python testing tool to use in the notebook.  `doctest`s are tests that live in strings and Python docstrings, and `>>>` indicates a place where a test is run.

[`doctest` discovers tests](https://docs.python.org/3/library/doctest.html#which-docstrings-are-examined) in the docstrings of functions, classes, modules, and a special dictionary `__test__`.

In [1]:
    def a_function_with_a_doctest(i): 
        """Converts `i` into a string representation
        
        >>> assert all(isinstance(
        ... a_function_with_a_doctest(object), str) for object in (range(10), 1))
        """
        return str(i)

`__doc__` the module docstring is tested.  

In [2]:
    __doc__ = """
    For the sake of example, the docstring for the module is created if it doesn't exist
    or appended to the original docstring.
    
    Doctest registers the statement below as a test.
    
    >>> assert True
    """

In [3]:
    if __name__ == '__main__': print(__import__('doctest').testmod())

TestResults(failed=0, attempted=2)


`__test__` is dictionary object that may contain named doctest-able objects like strings, functions, classes, and modules.  The key of the dictionary in the test name.

In [4]:
    __test__ = {
        'a_doc_test': """This is a docstring with tests that will be run.

    >>> assert True"""}

Conveniently, `__test__` is defined using a private namespace, but this means that a notebook author could easily transport tests within their notebooks and modules without any change to their syntactic choices.

## Unittest

[`unittest`](https://docs.python.org/3/library/unittest.html) is a buitlin unit testing framework for python.  It differs from `doctest` in that the tests are actualy python objects.

In [7]:
    from unittest import TestCase, main as _main

`unittest` collects classes subclasses as `TestCase`.

In [8]:
    class myTest(__import__('unittest').TestCase):
        """This is a unitest class with a docstring.  `doctest`"""
        
        def test_assertion(self): assert True

In [9]:
    def main():
        try: _main(argv=['discover'])
        except SystemExit: ...

In [10]:
    main()

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


The snippet below includes `doctest`s within `unittest`s.

In [11]:
    from doctest import DocTestSuite
    def load_tests(loader, tests, ignore):
        tests.addTests(DocTestSuite(__import__(__name__)))
        return tests


In [12]:
    main()

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


*We can observe that more tests were collection because the `load_tests` object exists.*

## pytest

[`pytest`](http://doc.pytest.org/) is one of the most common frameworks in python used for testing.  It has an extensive plugin framework and ecosystem that allows non experts to create robust testing frameworks.  [`pytest`](http://doc.pytest.org/) generally requires less syntax that traditional doctests especially when combined with [`pytest.fixtures`](https://www.google.com/search?q=pytest+fixtures&rlz=1C1AVFC_enUS803US803&oq=pytest+fixtures&aqs=chrome..69i57j0l5.3743j0j7&sourceid=chrome&ie=UTF-8).

`pytest` discovery is configurable, but by default tests beginning with `test_*` will be discovered.

In [13]:
    def test_thing(a: (range(10), 1)):
        assert isinstance(a_function_with_a_doctest(a), str)

### unittest, doctest, and pytest

`pytest` automated collections unittests and has options for doctesting.

### Running pytest in an IPython context

A notebook author will often use IPython magics and other IPython specific API's.  These applications require the test be run with an active IPython context.  Most examples of `pytest` run tests using vanilla pytest by invoking either `pytest` or `python -m pytest` at the command line.  In these situations, the IPython context is unavailable.

The code snippet below shares an application of running `pytest` through IPython using

    ipython -m doctest --
    
where any argument following `--` 

In [14]:
    import pytest

As stated, `pytest` discovery is configurable; notebooks are not python files so we must modify the way tests are found using [`pytest_collect_file`](https://docs.pytest.org/en/latest/example/nonpython.html).

In [15]:
    def pytest_collect_file(parent, path):
        if path.ext == ".ipynb": return pytest.Module(path, parent)

Run `pytest` and collect notebook files.

In [18]:
    def _run_pytest_discovery():
        pytest.main('--collect-only -p no:pytest-importnb 2018-07-31-Testing-notebooks.ipynb'.split(), [__import__(__name__)])

#### Interactive testing with `pytest`

> As far as we know, `pytest` can not run the current context in an IPython session.  A Pytest runner is required.


## Summary

* `doctest`s run tests in strings and __docstrings__ .
* `unittest`s run tests on objects and may include `doctest`.
* `pytest` runs tests on objects and may include `doctest` and `unittest` tests.

In general, we end a lot of our essays with `doctest` to promote better documentation and reuse during an interactive session.  `pytest` is applied to our files or collections of files.